In [29]:
!pip install icecream
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
!git clone https://github.com/illidanlab/personaGPT.git

fatal: destination path 'personaGPT' already exists and is not an empty directory.


In [ ]:
!pip install transformers
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
sys.path.append('/content/personaGPT/')

In [ ]:
import torch, os, pickle, time
import numpy as np
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 12 14:21:35 2020

@author: af1tang
"""
import os, torch, pickle
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoTokenizer, AutoModelWithLMHead

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# paths and configs
save_path = '/content/pc.t'
tokenizer_path = 'af1tang/personaGPT'
model_path = 'af1tang/personaGPT'
data_path = '/content/personaGPT/data'

def create_dir(directory):
    """create directory if not exists
    """
    if not os.path.exists(directory):
        os.makedirs(directory)

# initialize save folder
create_dir(save_path)

# global pretrained model and tokenizer
def load_from_pretrained():
    print("*"*50)
    print("Load from checkpoint")
    tokenizer = GPT2Tokenizer.from_pretrained('af1tang/personaGPT', 
                                            pad_token='<|endoftext|>', cls_token='<|cls|>',
                                            sep_token='<|sep|>')
    model = GPT2LMHeadModel.from_pretrained('af1tang/personaGPT')
    try:
        with open(os.path.join(opts.output_dir, 'pretrain_stats.pkl'), 'rb') as f:
            pretrain_stats = pickle.load(f)
        with open(os.path.join(opts.output_dir, 'train_stats.pkl'), 'rb') as f:
            train_stats = pickle.load(f)
    except: 
        print("Can't find training stats...")
        pretrain_stats, train_stats = None, None
    print("*"*50)

    tokenizer.add_special_tokens({'additional_special_tokens': ['<|start|>', '<|p1|>', '<|p2|>', '<|act|>']})
    model.resize_token_embeddings(len(tokenizer))
    return model.to(device), tokenizer, pretrain_stats, train_stats


model, tokenizer, pretrain_stats, train_stats = load_from_pretrained()
p1_tok, p2_tok, start_tok = tokenizer.encode('<|p1|>')[0], tokenizer.encode('<|p2|>')[0], tokenizer.encode('<|start|>')[0]

# new, action token
act_tok = tokenizer.encode('<|act|>')[0]

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 12 14:17:28 2020

@author: af1tang
"""
import torch, os, pickle, matplotlib.pyplot as plt
import torch.nn as nn, torch.nn.functional as F
from itertools import groupby

## Utils ##
flatten = lambda l: [item for sublist in l for item in sublist]
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
    
def to_data(x):
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()

def to_var(x):
    if not torch.is_tensor(x):
        x = torch.Tensor(x)
    if torch.cuda.is_available():
        x = x.cuda()
    return x

def process_conv(row, tokenizer, eos = True, make_flat=True):
    if eos:
        conv = list([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row])
    else: conv = list([tokenizer.encode(x) for x in row])
    if make_flat: conv = flatten(conv)
    return conv

def split_by_index(seq, sep):
    result = []
    for el in seq:
        result.append(el)
        if el == sep:
            yield result
            result = []
            
def filter_turn_indices(x):
    filtered = [[t[1] for t in list(g)] for k,g in groupby(list(enumerate(x)), lambda x: x[1]==tokenizer.eos_token_id) if not k]
    return filtered

def display_dialog_history(dialog_hx):
    for j, line in enumerate(dialog_hx):
        msg = tokenizer.decode(line)
        if j %2 == 0:
            print(">> User: "+ msg)
        else:
            print("Bot: "+msg)
            print()

In [ ]:
next(model.parameters()).device

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 12 11:40:29 2020

@author: af1tang
"""
from tqdm import tqdm
import torch, os, pickle
import torch.nn as nn, torch.nn.functional as F
import numpy as np, random
import pandas as pd

## Persona Preprocess ##
def preprocess_convai(filename):
    raw_data = open(filename).read().strip().split('\n')
    data, count = {}, 0
    curr_convo, curr_ps, curr_pt = [], [], []
    indices = []
    
    person_a = 'your persona'
    person_b = "partner's persona"
    with tqdm(total = len(raw_data)) as pbar:
        turn_count, ctx_count = 1,0 #init cycle
        for idx, line in enumerate(raw_data):
            if person_a in line[0:20]:
                if (turn_count != 0) and (len(curr_ps)>1 and len(curr_pt)>1 and len(curr_convo)>1):
                    if idx > 1:
                        if curr_convo[0] == '__SILENCE__' :
                            p1 = curr_ps; p2 = curr_pt; curr_convo = curr_convo[1:]
                        else:
                            p1 = curr_pt; p2 = curr_ps
                        data[count] = { 'inp': process_conv([curr_convo[0]], tokenizer),
                                        'labels': process_conv(curr_convo[1:],tokenizer), #to_data(torch.cat(curr_convo,dim=-1)[0]), 
                                       'p_src': process_conv(p1, tokenizer,make_flat=False), #to_data(torch.cat(curr_ps,dim=-1)[0]),
                                       'p_trg': process_conv(p2, tokenizer, make_flat=False)}#to_data(torch.cat(curr_pt,dim=-1)[0])}
                        count+=1
                    curr_convo, curr_ps, curr_pt = [], [], []
                    turn_count=0

                words = line.split()
                turn_id, words = int(words[0]), ' '.join(words[3:])
                curr_ps.append(words)

                ctx_count +=1
                assert ctx_count == turn_id
                
            elif person_b in line[0:20]:
                if (turn_count != 0) and (len(curr_ps)>1 and len(curr_pt)>1 and len(curr_convo)>1):
                    if idx > 1:
                        if curr_convo[0] == '__SILENCE__' :
                            p1 = curr_ps; p2 = curr_pt; curr_convo = curr_convo[1:]
                        else:
                            p1 = curr_pt; p2 = curr_ps
                        data[count] = { 'inp': process_conv([curr_convo[0]], tokenizer),
                                        'labels': process_conv(curr_convo[1:],tokenizer), #to_data(torch.cat(curr_convo,dim=-1)[0]), 
                                       'p_src': process_conv(p1, tokenizer,make_flat=False), #to_data(torch.cat(curr_ps,dim=-1)[0]),
                                       'p_trg': process_conv(p2, tokenizer, make_flat=False)}#to_data(torch.cat(curr_pt,dim=-1)[0])}
                        count+=1
                    curr_convo, curr_ps, curr_pt = [], [], []
                    turn_count=0
                words = line.split()
                turn_id, words = int(words[0]), ' '.join(words[3:])
                curr_pt.append(words)

                ctx_count +=1
                assert ctx_count == turn_id

                
            else:
                if ctx_count !=0:
                    turn_count = ctx_count *1 
                    ctx_count =0
                    indices.append(idx)
                        
                src_line, trg_line = line.split('\t')
                src_words = src_line.split()
                src_idx, src_line = src_words[0], ' '.join(src_words[1:])

                curr_convo.append(src_line) 
                curr_convo.append(trg_line)#turn)
                
                turn_count +=1
                assert turn_count == int(src_idx)
                
            pbar.update(1)
        
    return data

def convert_to_XY(old_data):
    data = []
    print("building training set...")
    for i in range(len(old_data)):
        p1 = [tokenizer.decode(p) for p in old_data[i]['p_src']]
        p2 = [tokenizer.decode(p) for p in old_data[i]['p_trg']]

        convo = old_data[i]['inp'] + old_data[i]['labels']
        dialog_hx = list(split_by_index(convo,tokenizer.eos_token_id))
        #if len(dialog_hx) < 30:
        dialog_hx = [line + [tokenizer.eos_token_id] for line in dialog_hx[:20]] # limit by max len of convo
        p1_ctx = tokenizer.encode(''.join(['<|p1|>'] + p1 + ['<|sep|>'] + ['<|start|>']))
        p2_ctx = tokenizer.encode(''.join(['<|p2|>'] + p2 + ['<|sep|>'] + ['<|start|>']))
        for t in range(len(dialog_hx)):
            x = dialog_hx[:t]
            y = dialog_hx[t]
            if t == 0:
                x = p1_ctx[:-1] 
                y = [p1_ctx[-1]] + y
            elif t %2 ==0:
                x = p1_ctx + flatten(x)
            else:
                x = p2_ctx + flatten(x)
            data.append((x,y))
    return data

def build_active_data():
    df = pd.read_csv(os.path.join(data_path, 'active_learning_data.csv'))
    X, y = df['context'].tolist(), df['response'].tolist()
    X, y = [tokenizer.encode(x) for x in X], [tokenizer.encode(yy) for yy in y]
    data = {'X':X, 'y':y}
    return data
    
    
if __name__ == '__main__':        
    val_data = preprocess_convai('/content/personaGPT/data/valid_both_original_no_cands.txt')
    val_data = convert_to_XY(val_data)
    with open('/content/personaGPT/data/val_data', 'wb') as f: pickle.dump(val_data, f)

In [ ]:
from icecream import ic

In [ ]:
!pip install bert_score sacrebleu
!pip install git+https://github.com/google-research/bleurt.git

In [ ]:
import evaluate
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")
sacrebleu = evaluate.load("sacrebleu")
bleurt = evaluate.load("bleurt", module_type="metric")
meteor = evaluate.load('meteor')

In [ ]:
avg_rouge = []
avg_bleu = []
avg_bert = []
avg_sbleu = []
avg_bleurt = []
avg_meteor = []

In [ ]:
with open('/content/personaGPT/data/val_data', 'rb') as f: data = pickle.load(f)
dataloader = DataLoader(data, batch_size=1, shuffle=True); del data
data_iter = iter(dataloader)
with torch.no_grad():
    eval_stats, total_steps, val_loss, val_f1_score = {}, 0, 0.0, 0.0
    model.eval()
    for i in tqdm(range(len(dataloader))):
        batch = next(data_iter)
        xx,yy = batch
        try:
            xx, yy = torch.stack(xx, -1).to(device), torch.stack(yy, -1).to(device)
        except:
            xx, yy = to_var(xx), to_var(yy)
        ## forward on new data batch
        dic_output = model(xx);
        past = dic_output['past_key_values']
        outp = model(yy, past_key_values=past, labels=yy)
        loss = outp[0]
        ytrue=np.array( filter_turn_indices(to_data(yy[...,1:].contiguous().view(-1)) ) )
        ytext = tokenizer.decode(torch.LongTensor(ytrue[0]))
        ypred=np.array( filter_turn_indices(to_data( outp[1][..., :-1, :].contiguous().topk(1)[1].view(-1)) ) ) 
        try:
          yptext = tokenizer.decode(torch.LongTensor(ypred[0]))
        except:
          yptext = "<pad>"

        # avg_rouge.append(rouge.compute(references=[ytext], predictions=[yptext])['rougeL'])
        avg_bleu.append(bleu.compute(predictions=[yptext], references=[ytext])['bleu'])
        avg_bert.append(bertscore.compute(predictions=[yptext], references=[ytext], lang='en')['f1'][0]) # 5m 
        avg_sbleu.append(sacrebleu.compute(predictions=[yptext], references=[ytext])['score']) # almost no time
        avg_bleurt.append(bleurt.compute(predictions=[yptext], references=[ytext])['scores'][0]) # half n hour
        avg_meteor.append(meteor.compute(predictions=[yptext], references=[ytext])['meteor']) # 10 minutes

        min_len = min(len(ypred), len(ytrue))
        hits = [set(ypred[i]).intersection(set(ytrue[i])) for i in range(min_len)]
        prec = [len(hits[i])/len(ypred[i]) for i in range(min_len)]
        rec = [len(hits[i])/len(ytrue[i]) for i in range(min_len)]
        f1 = np.mean([2*(prec[i]*rec[i])/(prec[i] + rec[i]+1e-3) for i in range(min_len)])
        val_f1_score += f1
        val_loss += loss.mean().item()
        total_steps +=1 
        #if total_steps%100 ==0: print("... %d out of %d"%(total_steps, len(dataloader)))
        
val_loss = val_loss / total_steps 
val_f1_score = val_f1_score / total_steps
perplexity = torch.exp(torch.tensor(val_loss)).item()
eval_stats = {'perplexity': perplexity,
              'loss': val_loss,
              'f1': val_f1_score,
              # 'rougeL': sum(avg_rouge)/len(avg_rouge),
              'bleu': sum(avg_bleu)/len(avg_bleu),
              'bertscore': sum(avg_bert)/len(avg_bert),
              'sacrebleu': sum(avg_sbleu)/len(avg_sbleu),
              'bleurt': sum(avg_bleurt)/len(avg_bleurt),
              'meteor': sum(avg_meteor)/len(avg_meteor)
              }
print("Done.")
print(eval_stats)